# <stan style="color:orange">Distribución espacial de las fibras y cálculo de sus distancias al electrodo </stan>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
#import pandas as pd
#from scipy.interpolate import interp1d
import random
from shapely import Polygon,Point
from shapely.plotting import plot_polygon, plot_points
#import geopandas as gpd

In [ ]:
import sys
!{sys.executable} -m pip install geopandas

## <span style="color: cyan"> Señal de entrada</stan>


Esto no es parte del modelo. Es una forma de generar la entrada al modelo. Esta es una forma medio pobre, voy a tener que buscarle una forma mejor. 

In [1]:
fs = 20000
#t1 = np.linspace(1/fs, 0.4, 8000)
t1=np.arange(1/fs,0.4+1/fs,1/fs)
fc = 2000
orden = 2
b, a = signal.butter(orden, fc/(fs/2))
x = np.zeros_like(t1)
x[3999] = 1
Vm = signal.lfilter(b, a, x)
Vm = Vm / np.max(Vm)
plt.plot(t1, Vm)
print(type(Vm))
print(t1,type(t1),len(t1))

NameError: name 'np' is not defined

In [ ]:
fs = 20000
t_estimulo = 0.4
t_inicio = 0.2
z = int(t_inicio*fs)

# Filtro
fc = 100
orden = 2

In [ ]:
fs = 20000
t_estimulo = 0.4
t_inicio = [0.1,0.3]
#z = int(t_inicio*fs)
fc = 100
orden = 2
t = np.arange(1/fs,t_estimulo+1/fs,1/fs)
b,a = signal.butter(orden,fc/(fs/2))
x = np.zeros_like(t)
x[100] = 1
x[3000] = 1
x[5000] = 1
Vm = signal.lfilter(b,a,x)
Vm = Vm/np.max(Vm)
plt.plot(t,Vm)

In [ ]:
fs = 20000
t_estimulo = 0.4
inicio = [0.1,0.3]
z = [int(x) for x in inicio]
fc = 100
orden = 2
t = np.arange(1/fs,t_estimulo+1/fs,1/fs)
b,a = signal.butter(orden,fc/(fs/2))
x = np.zeros_like(t)
x[100] = 1
x[500] = 1
Vm = signal.lfilter(b,a,x)
Vm = Vm/np.max(Vm)
plt.plot(t,Vm)

In [ ]:
z

In [ ]:
[int(x) for x in lista]

## <stan style="color: cyan"> Modelado del nervio y sus fibras </stan>

AGREGAR:
* Mejorar la distribución con coordenadas polares
* Permitir hacer otro tipo de distribución que no sea la uniforme
* Permitir ingresar manualmente la distribución de las fibras
* Ver qué onda con el espacio finito de las fibras (que no se sobrepongan una sobre otra) (ver si tiene sentido)
* Que el electrodo no sea un punto sino que tenga un tamaño, así la distancia de la fibra al electrodo sea la mínima distancia

In [ ]:
#Distribución uniforme


nu = 200 # cantidad de fibras
diametro = 250 #diametro del nervio
radio = diametro/2

fibras = np.zeros((nu,2))
np.random.seed(123)
#Distribución uniforme
for i,num in enumerate(fibras):
    length = np.random.uniform(0,radio**2)
    angle = np.pi * np.random.uniform(0, 2)
    x = np.sqrt(length) * np.cos(angle)
    y = np.sqrt(length) * np.sin(angle)
    fibras[i] = [x,y]

#print(fibras)

plt.figure(figsize = (5,5))
plt.plot(fibras[:,0],fibras[:,1],"r .")

In [ ]:
#Distribución Normal (De 0 a pi)

nu = 2000 # cantidad de fibras
diametro = 250 #diametro del nervio
radio = diametro/2

fibras = np.zeros((nu,2))


for i,num in enumerate(fibras):
    length = np.random.normal(0,radio)
    angle = np.pi * np.random.uniform(0, 1)
    #print(length,angle)
    while np.abs(length) > radio:
        length = np.random.normal(0,radio)
    #print("x", length, angle)
    x = length * np.cos(angle)
    y = length * np.sin(angle)
    fibras[i] = [x,y]

#print(fibras, len(fibras))


plt.figure(figsize = (5,5))
plt.plot(fibras[:,0],fibras[:,1],"r .")

In [ ]:
#Distribución Normal (De 0 a 2pi)

nu = 2000 # cantidad de fibras
diametro = 250 #diametro del nervio
radio = diametro/2

fibras = np.zeros((nu,2))

for i,num in enumerate(fibras):
    length = np.random.normal(0,radio)
    angle = np.pi * np.random.uniform(0, 2)
    while np.abs(length) > radio:
        length = np.random.normal(0,radio)
    x = length * np.cos(angle)
    y = length * np.sin(angle)
    # TODO así se carga fibra a fibra --> self.lista_fibras.append(Fibra(x,y))
    fibras[i] = [x,y]

plt.figure(figsize = (5,5))
plt.plot(fibras[:,0],fibras[:,1],"r .")

In [ ]:
#Ingreso manual de unicación de fibras

fibras = pd.read_csv("fibras.csv")

numero_fibras = len(fibras)

for i in range(0,numero_fibras):
    x = fibras["x"][i]
    y = fibras["y"][i]
    #self.lista_fibras.append(Fibra(x,y))


In [ ]:
# Calculo de distancia de cada fibra electrodo --------------------------------
dist = []
electrodo = [0,a] #ubicacion del electrodo
for item in puntitos:
    #dist.append(np.sqrt((item[0])**2 + (np.abs(a) + item[1])**2))
    dist.append(np.linalg.norm(item-electrodo))

dist = np.array(dist)

#print(dist,len(dist),type(dist))

#! Ver de cambiar la forma en la que creo dist y los puntos, así no sean primero listas


In [ ]:
# Más parámetros del modelo

nn = 2 #numero de nodos
den = 1000 #distancia entre nodos
dx = 12000 #distancia en micras al primer electrodo
dx1 = 16000 #distancia en micras al segundo electrodo

In [ ]:
#Prueba de distancia de un punto a una línea
punto1 = np.array([0,4])
punto2 = np.array([6,4])
punto3 = np.array([50,1])
d = np.linalg.norm(np.cross(punto2-punto1, punto1-punto3))/np.linalg.norm(punto2-punto1)
print(d)

In [ ]:
import numpy as np
from numpy.linalg import norm

A = np.array([0,4])
B = np.array([6,4])
P = np.array([10,0])

if np.arccos(np.dot((P-A)/norm(P-A), (B-A)/norm(B-A))) > np.pi/2:
    d = norm(P-A)
elif np.arccos(np.dot((P-B)/norm(P-B), (A-B)/norm(A-B))) > np.pi/2:
    d = norm(P-B)
else:
    d = norm(np.cross(B-A, A-P))/norm(B-A)

print(d)

## <stan style="color: cyan"> Copia de la entrada para cada fibra del nervio</stan>

In [ ]:
resp = [] #lo comienzo como una lista
for i in range(0,nu): #punto de mejora de entradas, aqui digo que es el mismo
    resp.append(Vm) 

#! Ver donde corcho vuelvo a usar esto porque ni idea

## <stan style="color:cyan "> Cálculo del factor mínimo de atenuación (Wmax) </stan>

In [ ]:
Wmax = np.zeros_like(dist)
for i,distancia in enumerate(dist):
    if (1-(distancia/40))<0:
        Wmax[i]=0.01
    else:
        Wmax[i]=1-(distancia/40)

#print(Wmax,len(Wmax),type(Wmax))

In [ ]:
range(5)

## <stan style="color:cyan">Función para calcular el factor de ponderación (pesosW en Matlab) </stan>

In [ ]:
d=nn*den
dx=500

x=np.array(list(range(0,200000,50))) #con 200mil hay 4mil datos
ww = np.zeros(len(x)) #tamaño 1x4mil
sizeW = (len(Wmax),len(x))
W = np.zeros(sizeW) #debería ser [1x4mil ...1x4mil] de tamaño Wmax(que es 200)

for i,v in enumerate(Wmax):
    for idx,num in enumerate(x):
        if num<dx:
            m = (num/d)-(dx/d)+v
        else:
            m = (-num/d)+(dx/d)+v
        if m<0:
            m = 0
        W[i,idx]=m

#print(W)

#df = pd.DataFrame(W).T
#df.to_excel(excel_writer ="C:/Users/aguir/Desktop/Prroyecto/test.xlsx")       

## <stan style="color:cyan">Cálculo de las velocidades de conducción nerviosa </stan>

Se realiza la asignación de diámetros. Promedio 5.5 micras y desviación estándar de 1.5 micras.
Las velocidades de conducción tienen una relación lineal con el diámetro de las fibras, y puede ser estimado multiplicandola por 7.6. <span style="color: red;">Buscar bien por qué usan ese número.</span>

In [ ]:
mu, sigma = 5.5 , 1.5
diametros = np.random.normal(mu,sigma,nu) #array con diámetros
#print(diametros,type(diametros),len(diametros))

In [ ]:
#Gráfica del histograma de diámetros

#plt.hist(diametros,20)
#plt.show()


In [ ]:
vel = np.zeros(len(diametros))
for d,n in enumerate(diametros):
    vel[d]=n*7.6

## <stan style="color:cyan">Función Cálculo del single fiber action potential (sum_SFAP en Matlab) </stan>

Desde aquí abajo comienza la función

In [ ]:
N = len(W) #cantidad de fibras del nervio
x=np.array(list(range(0,200000,50))) #vector con resolución de 50 micras, igual que en pesosW()

#Ver que estos valores, si yo los cambié arriba o que onda
dx = 12000
den = 1000

#Para conseguir dd
#TODO ver si se lo puede hacer más corto
dd1 = np.arange(nn,0,-1)
dd2 = np.arange(0,nn)
for i,n in enumerate(dd1):
    dd1[i]=dx-(n*den)
for i,n in enumerate(dd2):
    dd2[i]=dx+n*den
dd = np.concatenate((dd1,dd2), axis=0)
print(dd)

fss = 100000 #subfrecuencia de muestreo (para implementar retardos)

t11=np.arange(1/fss,t1[-1]+1/fss,1/fss) #vector de tiempo remuestrado usando fss
resp_pos = np.zeros((N,len(t11))) #inicialización de la matriz respuesta


Aquí sigue con el ciclo for 

In [ ]:
# CICLO FOR HORRIBLE ---------------------------------------------


#! Primero vuelvo a resp un array, ver de cambiarlo arriba
resp = np.array(resp)

for i in range(0,N):
    f = interp1d(t1,resp[i],fill_value="extrapolate")
    RESP = f(t11)
    xx = np.array([i for (i,val) in enumerate(W[i]) if val!= 0])
    d = x[xx[0]]
    reti = np.round(((d/1000)/(vel[i]*1000))*fss)
    ret = 0
    S = np.zeros((len(dd),len(t11)))
    #Comienza el segundo for :O 
    for j in range(len(dd)-1):
        f1 = interp1d(x,W[i],fill_value="extrapolate")
        aux2 = f1(dd[j])
        aux = RESP*dd[j]
        S[j][int(reti+ret+1):len(aux)] = aux[1:(len(aux)-int(ret+reti))]
        ret=np.round((den/1000)/(vel[i]*1000)*fss)+ret
    resp_pos[i]= sum(S)
sum_SFAP1 = sum(resp_pos)


        

In [ ]:
    def distribucion_normal(self,num_fibras,semilla = None):
        np.random.seed(semilla)
        radio = self.diametro/2
        for i in range(num_fibras):
            length = np.random.normal(0,radio)
            angle = np.pi * np.random.uniform(0, 2) #? de 0 a pi o 2pi? este de es 0 a 2pi
            while np.abs(length) > radio:
                length = np.random.normal(0,radio)
            x = length * np.cos(angle)
            y = length * np.sin(angle)
            self.fibras.append(Fibra(x,y))
        self.cantidad_fibras = num_fibras

In [ ]:
lista = [1,2,3,4,5]
arreglo = np.array(lista)
print(lista)
print(arreglo)

In [ ]:
lista = np.genfromtxt('C:/Users/aguir/Desktop/Prroyecto/fibras_2.csv', delimiter = ',')
print(lista,type ( lista))

In [ ]:
print(lista)

In [ ]:
for i in lista:
    print(type(float(i[0])),float(i[0]))


In [ ]:
plt.gca().add_patch(plt.Circle((1, 0.5), 0.2, edgecolor = 'black',facecolor = 'None'))
plt.axis('equal')
plt.show()

In [ ]:
radio = 250/2
num_fibras = 200

In [ ]:
longitud = np.random.uniform(0,radio**2, size = num_fibras)
angulo = np.pi * np.random.uniform(0,2,size = num_fibras)
x = np.sqrt(longitud) * np.cos(angulo)
y = np.sqrt(longitud) * np.sin(angulo)

In [ ]:
    def manual(self,lista_fibras):
        """Método con el cual se puebla el nervio con fibras de forma manual, ingresando una lista con las posiciones de las fibras.

        Args:
            lista_fibras (list): Lista con las posiciones de las fibras.
            num_fibras (int): Número de fibras con las que se pobló el nervio.

        Raises:
            OutOfBounds: Error por encontrar una fibra que supere los límites del nervio.
        """
        for f in lista_fibras:
            if not is_inside_puntoencirculo(self.radio,f[0],f[1]):
                

In [ ]:
a = np.genfromtxt('C:/Users/aguir/Desktop/Prroyecto/fibras_convacios.csv',delimiter = ',')

## <stan style="color : cyan"> Ubicación de nodos </stan>

In [ ]:
nn = 8
den = 5
xe = 10

In [ ]:
# ESTE SIRVE PARA TODOS
np.linspace(xe-((den/2) * (nn-1)),xe+((den/2) * (nn-1)),nn)

In [ ]:
nn = 9
den = 4
xe = 10

In [ ]:
# NO USAR
np.linspace(xe-(nn//den)*den,xe+(nn//den)*den,nn)

### <stan style="color: orange">1. Variar la distancia entre nodos en un porcentaje</stan>

In [ ]:
# ESTE SIRVE PARA TODOS
np.linspace(xe-((den/2) * (nn-1)),xe+((den/2) * (nn-1)),nn)

In [ ]:
np.array([1,10,100])*np.random.uniform(0.9,1.1)

In [ ]:
a = 10
var = 0
a = a + a*np.random.uniform(-var/100,var/100)
print(a)

In [ ]:
nn = 9
den = 4
xe = 10

In [ ]:
var = 10 #este será el porcentaje que va a variar

In [ ]:
den = den + den * np.random.uniform(-var/100,var/100)
print(den)
np.linspace(xe-((den/2) * (nn-1)),xe+((den/2) * (nn-1)),nn)

In [ ]:
#NO USO ESTO AL FINAL, USO EL DE ARRIBA
den = den * np.random.uniform(1-var/100,1+var/100)
print(den)
np.linspace(xe-((den/2) * (nn-1)),xe+((den/2) * (nn-1)),nn)

### <stan style="color : orange"> 2. Correr la ubicación de las fibras en un porcentaje </stan>

In [ ]:
# ESTE SIRVE PARA TODOS
np.linspace(xe-((den/2) * (nn-1)),xe+((den/2) * (nn-1)),nn)

In [ ]:
lista = np.array([1,10,100])

In [ ]:
lista + 1 #asi nomas le sumo un numero a todos

In [ ]:
lista + np.random.uniform(-1,1) #aumo o quito numero aleatorios a toda la lista

In [ ]:
nn = 9
den = 4
xe = 10

In [ ]:
mov = 1 #+- la cantidad que se quiera mover

In [ ]:
desp = np.random.uniform(-mov,mov)
np.linspace(xe-((den/2) * (nn-1))+desp,xe+((den/2) * (nn-1))+desp,nn)

In [ ]:
np.random.uniform(1,1)

### <stan style="color : orange"> 3. Unir variación y desplazamiento </stan>

In [ ]:
# FORMA GENERAL
np.linspace(xe-((den/2) * (nn-1)),xe+((den/2) * (nn-1)),nn)

In [ ]:
nn = 6
den = 2
xe = 10

In [ ]:
movimiento = 2
variacion = 10

In [ ]:
den = 2
desplazamiento = np.random.uniform(-movimiento,movimiento)
den = den + den * np.random.uniform(-variacion/100,variacion/100)
print(den,desplazamiento)

In [ ]:
np.linspace(xe-((den/2) * (nn-1))+desplazamiento,xe+((den/2) * (nn-1))+desplazamiento,nn)

In [ ]:
def funcion(param_a,param_b):
    pass

def otra_funcion(miparametro,tuparametro=0):
    a = miparametro * tuparametro
    funcion(param_b = 5,param_a = miparametro) #aqui tiene que ir en el mismo orden 

otra_funcion(5)

### <stan style = "color : cyan">Quitar elementos de un array con una condición</stan>

In [ ]:
a = np.array([-2,-3,0,1,2,4])


In [ ]:
a = a[a > 0]

In [ ]:
a

### <stan style = "color : cyan">Numpy random normal</stan>

In [ ]:
mu = 0
sigma = 0
np.random.normal(mu,sigma)

In [ ]:
np.random.normal(0,5,5)

### <stan style = "color : cyan">Numpy random uniform</stan>

In [ ]:
np.random.uniform(0,1,5)

In [ ]:
longitud_fibra = 50000
distancia_nodos = 10000
variacion = 1000
desplazamiento = 0

In [ ]:
arr = np.arange(-longitud_fibra/2,longitud_fibra*1.5 +1,distancia_nodos)
var = np.random.uniform(-variacion,variacion,len(arr)) + desplazamiento

In [ ]:
arr = np.arange(-longitud_fibra/2,longitud_fibra*1.5 +1,distancia_nodos)
var = np.random.normal(0,variacion,len(arr)) + desplazamiento

In [ ]:
arr

In [ ]:
var

In [ ]:
arr + var

In [ ]:
b = arr + var

In [ ]:
b

In [ ]:
b = b[(b>0) & (b<50000)]

In [ ]:
np.random.uniform(-1,1)

In [ ]:
a = np.array([1,3,2,2,6,8])

In [ ]:
a = [1,2,3,4]

In [ ]:
del a[-1]

In [ ]:
a

### <stan style = "color : cyan ">Dibujar con shapely  </stan>

In [ ]:
#coordenadas = [(1,1),(1,-1),(-1,-1),(-1,1)]
coord = Polygon([(1,1),(1,-1),(-1,-1),(-1,1)])
plot_polygon(coord,color = 'red', facecolor='None',edgecolor = 'black')
#plot_points(coord)
plt.show()

In [ ]:
cuadrado = Polygon([(0,0),(10,0),(10,10),(0,10)])
plot_polygon(cuadrado, facecolor='None', edgecolor='red')
coord = Polygon([(1,1),(1,-1),(-1,-1),(-1,1)])
plot_polygon(coord, add_points = False,facecolor='None', edgecolor = 'red')
plt.show()

In [ ]:
fig = plt.figure(1, dpi=90)
circ = Point((0,0)).buffer(1)
#plot_points(circ)
plot_polygon(circ, add_points=False,edgecolor = 'black',facecolor='None')

In [ ]:
fig.grid(False)

In [ ]:
plot_points(Point(0,0))

In [ ]:
shape = Point(0,0).buffer(0)
print(shape)


In [ ]:
polygon1 = Polygon([(0,5),
                    (1,1),
                    (3,0),
                    ])

 p = gpd.GeoSeries(polygon1)
 p.plot()
 plt.show()

In [ ]:
punto = Point(0,0)
circulo = Point(0,0).buffer(2)
pol = Polygon([(1,1),(1,-1),(-1,-1),(-1,1)])

In [ ]:
p = gpd.GeoSeries(punto)
p.plot(edgecolor = 'red', facecolor = 'None')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.plotting import plot_polygon

fig = plt.figure(1, dpi=90)

a = Point(1, 1).buffer(1.5)
b = Point(2, 1).buffer(1.5)

# 1
#ax = fig.add_subplot(121)
#ax.grid(True)
ax = plt.axes()
#ax.grid(True)
#plot_polygon(a, ax=ax, add_points=False)
plot_polygon(a,ax=ax,add_points=False)
#plot_polygon(b, ax=ax, add_points=False)

#c = a.intersection(b)
#plot_polygon(c, ax=ax, add_points=False)

#ax.set_title('a.intersection(b)')


#2
#ax = fig.add_subplot(122)
#
#plot_polygon(a, ax=ax, add_points=False)
#plot_polygon(b, ax=ax, add_points=False)
#
#c = a.symmetric_difference(b)
#plot_polygon(c, ax=ax, add_points=False)
#
#ax.set_title('a.symmetric_difference(b)')


#plt.show()

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.plotting import plot_polygon

a = Point(1, 1).buffer(1.5)
b = Point(2, 1)
x,y = a.exterior.xy
w,z = b.x,b.y

plt.axis('equal')
plt.plot(x,y)
plt.plot(w,z)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.plotting import plot_polygon


x,y = Point(1, 1).buffer(10).exterior.xy

plt.axis('equal')
plt.plot(x,y)

plt.show()

In [ ]:
lista = ['e1','e2','e3']

for e in lista:
    index = lista.index(e)
    print(index)


In [ ]:
d = np.random.normal(2,0)
print(d)

In [ ]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])

In [ ]:
a[:,3]

In [ ]:
np.array([[1,2,3],[4,5,6],[7,8,9]])[:,1]

In [ ]:
[[73.23958313,3.92731714,2.5],[9.90834512,-38.99651803,2.5],[-57.91269168,39.38951769,2.5]]

In [ ]:
a = np.array([[ 2.52311858e+01, -8.11397542e+01,  5.00000000e+00],
       [ 4.68025803e+01,  3.84263151e+01,  5.00000000e+00],
       [-7.49968319e+01,  9.04491870e+01,  5.00000000e+00],
       [-1.05791373e+02,  3.81325669e+01,  5.00000000e+00],
       [-2.71689366e+01, -7.73837089e+01,  5.00000000e+00],
       [-1.13240664e+01,  1.14682326e+02,  5.00000000e+00],
       [ 6.92916183e+01,  6.09061185e+00,  5.00000000e+00],
       [ 6.61862081e+01, -7.88947501e+01,  5.00000000e+00],
       [ 8.77197791e+01,  3.76309822e+01,  5.00000000e+00],
       [-7.54855225e+01, -7.83548324e+01,  5.00000000e+00],
       [-8.09183769e+01,  1.65342710e+01,  5.00000000e+00],
       [-6.99526161e+01, -3.68794459e+01,  5.00000000e+00],
       [-6.99146626e+01,  5.65192187e+01,  5.00000000e+00],
       [ 9.64513679e+01, -1.82731042e+01,  5.00000000e+00],
       [-1.07632461e+02, -4.61583003e+01,  5.00000000e+00],
       [ 8.34964956e+01, -3.26494592e+01,  5.00000000e+00],
       [-4.34147676e+01,  7.62816128e+01,  5.00000000e+00],
       [ 5.17527933e+01, -7.40475129e+01,  5.00000000e+00],
       [-4.13220670e+01, -5.16027760e+01,  5.00000000e+00],
       [ 4.28629419e+01,  9.34062130e+01,  5.00000000e+00],
       [-1.34884898e+01,  9.71445504e+01,  5.00000000e+00],
       [-7.41548364e+01, -4.64176393e+01,  5.00000000e+00],
       [-3.47172374e+01, -8.46711226e+01,  5.00000000e+00],
       [ 2.00431464e+01, -7.22293318e+01,  5.00000000e+00],
       [ 5.39503167e+00,  1.10591914e+02,  5.00000000e+00],
       [ 5.70455459e+01, -5.68794552e+01,  5.00000000e+00],
       [-6.12041635e+01, -7.02271503e+01,  5.00000000e+00]])

In [ ]:
np.savetxt("fibras_condiametro.csv", a, delimiter=",")

In [ ]:
b = a[:,(0,1)]

In [ ]:
np.savetxt("fibras_sindiametro.csv", b, delimiter=",")

### <stan style="color: cyan"> Borro los metodos manual y desde archivos separados</stan>

In [ ]:
    def manual(self,diametro_fibra,lista_fibras):
        """Método con el cual se puebla el nervio con fibras de forma manual, ingresando una lista con las posiciones de las fibras.

        Args:
            diametro_fibra (float): Diámetro de las fibras. 
            lista_fibras (list): Lista con las posiciones de las fibras.

        Raises:
            OutOfBounds: Error por encontrar una fibra que supere los límites del nervio.
        """
        f = [Fibra(x,y,diametro_fibra) for x,y in lista_fibras if is_inside_puntoencirculo(self.radio,x,y,self.centro_nervio_x,self.centro_nervio_y)]
        if len(f)!=len(lista_fibras):
            raise OutOfBounds ("Se encontró por lo menos una fibra fuera de los límites del nervio.")        
        self.fibras= f
        self.cantidad_fibras = len(lista_fibras)

    def manual_condiametro(self,lista_fibras):
        """Método con el cual se puebla el nervio de forma manual, a partir de un array de 3 columnas: coordenadas x, coordenadas y, diámetros de las fibras.

        Args:
            lista_fibras (list): Lista con la información de las fibras.

        Raises:
            OutOfBounds: Al detectar por lo menos una fibra fuera de los límites del nervio.
        """
            
        f = [Fibra(x,y,d) for x,y,d in lista_fibras if is_inside_circencirc(d/2,x,y,self.radio,self.centro_nervio_x,self.centro_nervio_y)]
        if len(f)!=len(lista_fibras):
            raise OutOfBounds ("Se encontró por lo menos una fibra fuera de los límites del nervio.")        
        self.fibras= f
        self.cantidad_fibras = len(lista_fibras)

In [ ]:
    def desde_archivo(self,diametro_fibra,ubicacion_archivo):
        """Método con el cual se carga un archivo con ubicación de fibras y se puebla el nervio con ellas.

        Args:
            diametro_fibra (float): Diámetro de las fibras en micrómetros.
            ubicacion_archivo (str):Dirección donde se encuentra el archivo csv con la ubicación de las fibras.

        Raises:
            LoadingError: Error de carga del archivo.
            ValueError: Se levanta cuando el archivo tiene un elemento no numérico.
            OutOfBounds: Se levanta cuando se encuentra una fibra posicionada fuera del diámetro del nervio.
        """
        try:
            lista_fibras = np.genfromtxt(ubicacion_archivo, delimiter = ',')
        except Exception as e:
            raise LoadingError('Fallo en carga de archivo.') from e
        
        if not np.isnan(sum(lista_fibras)).any() and np.issubdtype(lista_fibras.dtype, np.number):
            self.manual(diametro_fibra,lista_fibras)
        else:
            raise ValueError ("La lista contiene caracteres que no son números.")
    
    def desde_archivo_condiametro(self,ubicacion_archivo):
        """Método con el cual se puebla un nervio a partir de la carga de un archivo con la información de las fibras, las cuales deben organizarse en 3 columnas: coordenadas x, coordenadas y, diámetros de las fibras.

        Args:
            ubicacion_archivo (str):Dirección donde se encuentra el archivo csv con la información de las fibras.

        Raises:
            LoadingError: Error de carga del archivo.
            ValueError: Se levanta cuando el archivo tiene un elemento no numérico.
            OutOfBounds: Se levanta cuando se encuentra una fibra posicionada fuera del diámetro del nervio.
        """
        try:
            lista_fibras = np.genfromtxt(ubicacion_archivo, delimiter = ',')
        except Exception as e:
            raise LoadingError('Fallo en carga de archivo.') from e

        if not np.isnan(sum(lista_fibras)).any() and np.issubdtype(lista_fibras.dtype, np.number):
            self.manual_condiametro(lista_fibras)   
        else:
            raise ValueError ("La lista contiene caracteres que no son números.")

### <stan style="color : cyan">Np column stack</stan>

In [ ]:
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.array([7,8,9])
matriz = np.column_stack((a,b,c))
print(matriz)

### <stan style="color : cyan">Métodos uniformes que borro</stan>

In [ ]:
# Uniforme con superposición y diámetro fijo    
def distribucion_uniforme(self,num_fibras,semilla,diametro_fibra):
        """Método con el cual se puebla el nervio con fibras con distribución uniforme.
        El diámetro de las fibras es el mismo para todas las fibras con este método.

        Args:
            num_fibras (int): Número de fibras que tendrá el nervio.
            semilla (int, optional): Número de la semilla. Por defecto es None.
            diametro_fibra (float): Diámetro de la fibra en micrones. Por defecto es 0, si se trata de fibras puntuales.
        """
        #Tiene centro en (0,0), implementar el traslado 
        np.random.seed(semilla)
        longitud = np.random.uniform(0,self.radio**2, size = num_fibras)
        angulo = np.pi * np.random.uniform(0,2,size = num_fibras)
        x = np.sqrt(longitud) * np.cos(angulo)
        y = np.sqrt(longitud) * np.sin(angulo)
        matriz = np.column_stack((x,y))
        self.fibras = [Fibra(x,y,diametro_fibra) for x,y in matriz]
        self.cantidad_fibras = num_fibras

In [ ]:
#Uniforme sin superposición con diámetro fijo
def uniforme_sin_superposicion(self,num_fibras,semilla,diametro_fibra):
        """Método para poblar el nervio con el cual se evita la superposición de fibras entre sí, siendo contenidas dentro de los límites del nervio.

        Args:
            num_fibras (int): Número de fibras que tendrá el nervio.
            semilla (int, optional): Número de la semilla. Por defecto es None.
            diametro_fibra (float): Diámetro de la fibra en micrones.
        """
        np.random.seed(semilla)

        if diametro_fibra == 0 :
            raise InputError("Para este método es necesario ingresar un diámetro de fibras distinto a cero.")
 
        lista = []
        while len(lista) < num_fibras:
            choque = False
            length = np.random.uniform(0,self.radio**2)
            angle = np.pi * np.random.uniform(0, 2) 
            x = np.sqrt(length) * np.cos(angle)
            y = np.sqrt(length) * np.sin(angle)

            for a,b in lista:
                if distancia_entre_puntos(x,y,a,b) < diametro_fibra:
                    choque = True
            if choque is False and is_inside_circencirc(diametro_fibra/2,x,y,self.radio,self.centro_nervio_x,self.centro_nervio_y):
                lista.append([x,y])
        self.fibras = [Fibra(x,y,diametro_fibra) for x,y in lista]
        self.cantidad_fibras = num_fibras

### <stan style = "color : orange "> Pesos longitudinales lineal </stan>

In [ ]:
dist = [1,2,3,7,4,2,8,9]
b = 5

In [ ]:
[ (1-(x)/b)  if (x>=0 and x<b) else 0 for x in dist]

In [ ]:
lista = [10,10,10,10,10,10,10,10,10,10,10,10,10]
pos = list(range(1,21))
pesos = []
z = 8
d = 5
a = 1

pesos = [ 1-(abs(x-z)/d) if (x>z-d and x<z-a) or (x>z+a and x<z+d) 
         else 1 if (x>=z-a and x<=z+a) 
         else 0 
         for x in pos ]


In [ ]:
pesos

In [ ]:
list(range(1,21))

In [ ]:
def funcion_trapecio(x,z,d,a):
    if (x>z-d-a and x<z-a):
        return (x-z+d+a)/d
    elif (x>z+a and x<z+a+d):
        return (z+a+d-x)/d
    elif (x>=z-a and x<=z+a):
        return 1
    else:
        return 0

In [ ]:
def funcion_gaussiana2(x,z,d,a,mu):
    factor = 1/ (1/((d/3)*np.sqrt(2*np.pi)))
    
    if (x>z-d-a and x<z-a):
        coeficiente = 1/((d/3)*np.sqrt(2*np.pi))
        exponente = -(x - (mu -a))**2 / (2 * (d/3)**2)
        return factor*coeficiente*np.exp(exponente)
    elif (x>z+a and x<z+a+d):
        coeficiente = 1/((d/3)*np.sqrt(2*np.pi))
        exponente = -(x - (mu +a))**2 / (2 * (d/3 -a)**2)
        return factor*coeficiente*np.exp(exponente)
    elif (x>=z-a and x<=z+a):
        return 1

In [ ]:
def funcion_gaussiana(x,z,d,a,mu):
    coeficiente = 1/((d/3)*np.sqrt(2*np.pi))
    exponente = -(x - mu)**2 / (2 * (d/3)**2)
    if (x>z-d-a and x<z-a):
        return coeficiente*np.exp(exponente)
    elif (x>z+a and x<z+a+d):
        return coeficiente*np.exp(exponente)
    elif (x>=z-a and x<=z+a):
        return 1

In [ ]:
def funcion_gaussiana3(x,z,d,a):
    coeficiente = 1/((d/3)*np.sqrt(2*np.pi))
    factor = 1/coeficiente
    if (x<z-a):
        exponente = -(x - (z -a))**2 / (2 * (d/3)**2)
        return factor*coeficiente*np.exp(exponente)
    elif (x>z+a):
        exponente = -(x - (z +a))**2 / (2 * (d/3)**2)
        return factor*coeficiente*np.exp(exponente)
    elif (x>=z-a and x<=z+a):
        return 1
    else:
        return 0

In [ ]:
plt.plot(pos,y,'r .')

In [ ]:
plt.plot(pos,y)

In [ ]:
y

In [ ]:
def funcion_constante(x,z,d,a,valor):
    if x>=z-d-a and x<=z+d+a:
        return valor
    else:
        return 0

In [ ]:
pos = list(range(1,101))
y = [funcion_constante(x=x,z=50,d=20,a=5,valor = 50) for x in pos]

In [ ]:
plt.plot(pos,y,'r .')

In [ ]:
plt.plot(pos,y)

In [ ]:
y

In [ ]:
a = np.array([2,2,2])
b = np.array([[2,2,2,2],[1,1,1,1,1],[3,3,3]],dtype=object)

In [ ]:
lista_enteros = [1, 2, 3]
lista_listas = [[4, 5], [6, 7], [8, 9]]

resultado = []

for i in range(len(lista_enteros)):
    for j in range(len(lista_listas[i])):
        resultado.append(lista_enteros[i] * lista_listas[i][j])
        
print(resultado)

In [ ]:
lista_enteros = [1, 2, 3]
lista_listas = [[4, 0], [6, 7], [8, 9]]

resultado = []

for i in range(len(lista_enteros)):
    resultado_fila = []
    for j in range(len(lista_listas[i])):
        resultado_fila.append(lista_enteros[i] * lista_listas[i][j])
    resultado.append(resultado_fila)
        
print(resultado)

In [ ]:
lista_enteros = [1, 0, 3]
lista_listas = [[4, 0], [6, 7], [8, 9]]

resultado = [[entero * elemento for elemento in lista] for entero, lista in zip(lista_enteros, lista_listas)]

print(resultado)

In [ ]:
lista1 = [1, 1, 3, 1, 5]
lista2 = [0, 0, 0, 4, 0]

lista_filtrada = [lista1[i] for i, valor in enumerate(lista2) if valor == 0]

print(lista_filtrada)

In [ ]:
lista1 = [1, 1, 3, 9, 5] # lista1 = self.nervio.obtener_lista_fibras
lista2 = [0, 2, 0, 4, 2] # lista2 = self.electrodos[0].wmax

nueva_lista = [lista1[i] for i, valor in enumerate(lista2) if valor != 0]

print(nueva_lista)

In [ ]:
n = 5
m = 7

# Creamos la lista de tamaño n
lista_n = []
for i in range(n):
    # Creamos la lista de tamaño m con todos los elementos iguales a cero
    lista_m = [0] * m
    # Cambiamos el valor en la posición x a uno
    x = 3 # puedes cambiar este valor a cualquier posición que desees
    lista_m[x] = 1
    # Añadimos la lista_m a la lista_n
    lista_n.append(lista_m)

# Imprimimos la lista_n
print(lista_n)

In [ ]:
t_inicio = 5
f_muestreo = 2000
t_estimulo = 10
cant_muestras = f_muestreo*t_estimulo
posicion = t_inicio*f_muestreo
duracion = 0
num_fibras = 5


estimulo = np.array([np.array([1 if (j >= posicion and j<= (posicion+duracion)) else 0 for j in range(cant_muestras)]) for i in range(num_fibras)])

In [ ]:
np.savetxt('estimulo.csv',estimulo, delimiter=',')

In [ ]:
for i in range(len(estimulo)):
    plt.plot(estimulo[i])
plt.show()

In [ ]:
type(np.array([1,2]))

In [ ]:
a = 10
c = 3
signal = [0 for x in range(a)]
est = [1,1,1,1,1]
b = len(est)
print(signal)

In [ ]:
signal[c-1:c+b] = est
signal = signal[0:a]
print(signal)

In [ ]:
est[0:b-(a-c)+1]

In [ ]:
signal[c:a]

In [ ]:
m = 10 # Tamaño de la lista de ceros
n = 9 # Tamaño de la lista con los nuevos valores
i = 2 # Posición desde la cual se deben reemplazar los valores

# Lista de ceros
lista_ceros = [0]*m

# Lista con los nuevos valores
lista_nuevos_valores = [1, 2, 3, 4, 5]

# Reemplazo de valores
if n>=m-i:
    for j in range(m-i):
        lista_ceros[i+j] = lista_nuevos_valores[j]
else:
    for j in range(n):
        lista_ceros[i+j] = lista_nuevos_valores[j]

print(lista_ceros)

In [ ]:
def reemplazar_lista(lista_ceros, lista_numeros, posicion):
    m = len(lista_ceros)
    n = len(lista_numeros)
    i = posicion
    
    if n > m - i:
        n = m - i
    
    for j in range(n):
        lista_ceros[i+j] = lista_numeros[j]
    
    return lista_ceros

In [ ]:
def reemplazar_lista(lista_ceros, lista_numeros, posicion):
    m = len(lista_ceros)
    n = len(lista_numeros)
    i = posicion
    
    if n > m - i:
        n = m - i
    
    lista_ceros[i:i+n] = lista_numeros[:n]
    
    return lista_ceros

In [ ]:
lista_ceros = [0]*10
lista_numeros = [1] * 5
posicion = 0
reemplazar_lista(lista_ceros, lista_numeros, posicion)

In [ ]:
a = [1,2,'holi']

In [ ]:
a = np.array([1,2,3,4])

In [ ]:
np.array(a)

In [ ]:
b = np.tile(a,(5,1))

In [ ]:
b

In [ ]:
arr = np.array([1,'hola',1])

In [ ]:
arr = np.array(["Hola", "mundo", "!"])

In [ ]:

if arr.dtype.kind in np.typecodes["AllInteger"] + np.typecodes["Float"]:
    print("El array contiene valores numéricos")
else:
    print("El array no contiene valores numéricos")

In [ ]:
arr = np.array([1, 2.2, 3, 4, 5])
if np.issubdtype(arr.dtype, np.number):
    print("El array contiene valores numéricos")
else:
    print("El array no contiene valores numéricos")

In [ ]:

# Cargar datos desde el archivo CSV
datos = np.loadtxt("array.csv", delimiter=",")

# Imprimir los datos cargados
#print(datos)

# Imprimir la forma de los datos (número de filas y columnas)
print(datos.shape)

In [ ]:
a = np.zeros((5,20))

In [ ]:
len(a[0][2:5])

In [ ]:
a[0][2:5]

In [ ]:
arr = np.empty((2,5))

In [ ]:
arr = np.array([[1,'hola',1,1],[2,2,2,2],[3,3,3,3]])

In [ ]:
arr

In [ ]:
def is_numeric_array(arr):
    return np.isnan(sum(arr)).any()

In [ ]:
is_numeric_array(arr)

In [ ]:
def is_numeric_array(arr):
    return  np.issubdtype(arr.dtype, np.number)

def is_numeric_nested_array(arr):
    vfunc = np.vectorize(is_numeric_array)
    return vfunc(arr).all()

In [3]:
a = np.array([1,1,1])
b = np.zeros

In [4]:
np.vstack((a,b))

array([[1., 1., 1.],
       [0., 0., 0.]])

In [ ]:

arr = np.array([1, 2, 3])  # array conocido
n = 4  # número de copias

new_arr = np.repeat([arr], n, axis=0)  # crear un array con n copias de arr

print(new_arr,type(new_arr[0]))

In [ ]:
for i in new_arr:
    i = i/max(i)
    print(i)

In [ ]:
new_arr

In [ ]:
# Creamos un numpy array con varios arrays dentro
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Encontramos el valor máximo en cada array y dividimos cada array por su valor máximo
arr_norm = arr / np.amax(arr, axis=1, keepdims=True)

print(arr_norm)

In [ ]:
def insertar_estimulo(self,estimulo,posicion,fm,tiempo_registro,velocidad):
    total_muestras = tiempo_registro * fm
    muestras_estimulo = len(estimulo)
    vector = np.zeros(total_muestras)
    
    inicio = int((posicion/(velocidad*1000000)) * fm) #retardo
    
    if muestras_estimulo > total_muestras - inicio:
        muestras_estimulo = total_muestras-inicio
    
    vector[inicio:inicio+muestras_estimulo] = estimulo[:muestras_estimulo]
    
    

In [ ]:
a = np.array([2,2,2,2,2])
b = np.array([5,5,5,5,6])

In [ ]:
a = np.ones((2,2))

In [ ]:
a/2

In [ ]:
# Lo que borré

    # def desde_archivo(self,diametro_fibra,ubicacion_archivo,alfa):
    #     """Función para poblar un nervio, a partir de un archivo csv.

    #     Args:
    #         diametro_fibra (float): Diámetro que se le dará a la fibra, si es que lista_fibras no tuviera una columna con esta información.
    #         ubicacion_archivo (str): String con la ubicación del archivo csv.
    #         alfa (float, optional): Valor necesario para calcular la velocidad de conducción de las fibras.

    #     Raises:
    #         LoadingError: Error que se levanta cuando no se puede cargar el archivo.
    #         ValueError: Error que se levanta cuando se detecta que algún caracter ingresado en el archivo no es numérico.
    #     """
    #     try:
    #         fibras = np.genfromtxt(ubicacion_archivo, delimiter = ',')
    #     except Exception as e:
    #         raise LoadingError('Fallo en carga de archivo.') from e
        
    #     if not np.isnan(sum(fibras)).any() and np.issubdtype(fibras.dtype, np.number):
    #         self.manual_final(diametro_fibra=diametro_fibra,lista_fibras = fibras, alfa = alfa)   
    #     else:
    #         raise ValueError ("La lista contiene caracteres que no son números.")
 
    # def setear_diametros_fibras(self,diametro,variacion): 
    #     """Método que permite setear los diámetros en las fibras.

    #     Args:
    #         diametro (float): Diámetro de la fibra.
    #         variacion (float): Variación del diámetro. Sigma de la función normal.
    #     """
    #     radio = diametro/2
    #     for f in self.fibras:
    #         f.radio = np.random.normal(radio,variacion)
    
    # def setear_velocidad_conduccion(self, velocidad):
    #     """Método que permite setear la velocidad de conducción de las fibras. 

    #     Args:
    #         velocidad (float): Velocidad de conduccción de las fibras.
    #         alfa (float, optional): Valor necesario para calcular la velocidad de conducción. Por defecto es None.
    #     """
    #     # if velocidad is None:
    #     #     velocidad = 2 * self.fibras[0].radio * alfa
    #     # for f in self.fibras:
    #     #     f.velocidad = velocidad
    #     # self.velocidad_fibras = velocidad
        
    #     if velocidad is not None:
    #         for f in self.fibras:
    #             f.velocidad = velocidad

In [18]:
# Crear el array original
array_original = np.array([np.ones(3),np.ones(3),np.ones(3)]),[np.ones(3),np.ones(3)],[np.ones(3),np.ones(3),np.zeros(3)]

# Calcular la suma por fila
suma_por_fila = np.sum(array_original, axis=1)

# Imprimir el resultado
print(suma_por_fila)

AxisError: axis 1 is out of bounds for array of dimension 1

In [15]:
array_original = np.array([np.ones(3),np.ones(3),np.ones(3)]),[np.ones(3),np.ones(3)],[np.ones(3),np.ones(3),np.zeros(3)]
print(array_original)

(array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]]), [array([1., 1., 1.]), array([1., 1., 1.])], [array([1., 1., 1.]), array([1., 1., 1.]), array([0., 0., 0.])])


In [20]:
texto = 'Este es un texto largo que quiero imprimir \
en varias líneas utilizando la barra invertida. \
De esta forma, puedo separar el texto en diferentes líneas \
para que sea más fácil de leer.'
print(texto)

Este es un texto largo que quiero imprimir en varias líneas utilizando la barra invertida. De esta forma, puedo separar el texto en diferentes líneas para que sea más fácil de leer.
